In [175]:
#theano imports
%env THEANO_FLAGS="device=gpu2"
import lasagne
import theano
import theano.tensor as T
import sys
sys.setrecursionlimit(100000)
floatX = theano.config.floatX

env: THEANO_FLAGS="device=gpu2"


In [176]:
from nltk.stem.snowball import SnowballStemmer

In [33]:
#!pip install gensim --upgrade --target=/anaconda3/lib/python3.5/site-packages

In [34]:
import numpy as np

In [58]:
stemmer = SnowballStemmer("russian")

In [182]:
import json
questions = json.load(open('question_my.json'))

In [183]:
questions[0]['answers']

['Насколько может быть сильным самовнушение?',
 ['Психология', 'Мозг'],
 [['кандидат психологических наук, доктор философии, независимый консультант по организационному развитию и персональной эффективности руководителей',
   'Сила внушения может быть очень большой. Всем хорошо известны случаи, когда группы сектантов внушали себе неизбежность прихода конца света. При этом они доводили себя до такого состояния, что поджигали себя и сгорали заживо, то есть самовнушение, усиленное групповыми эффектами, может быть летальным. В более мягких случаях, например в спорте, способность к самовнушению позволяет выходить на старт в оптимальном состоянии и выигрывать соревнования или, наоборот, перегорать заранее, зажиматься и проигрывать еще до схватки. Тело и мозг, физиология и эмоции — взаимосвязанные части системы под названием «человек». Именно поэтому такие инструменты, как аутогенная тренировка, медитация, различные дыхательные практики, — всё это есть не что иное, как различные формы самовну

In [63]:
from tqdm import tqdm

In [75]:
import re
from collections import Counter

In [191]:
def process_textblob(txt):
    txt = txt.lower()
    txt = re.subn('<[a-z\s]*>', '', txt)[0]
    #txt = re.sub('http\w*\s', '', txt)
    txt = re.subn('[^А-Яа-яA-Za-z\-\.\,\s]', '', txt)[0]
    r = re.split('(\s|\-|\.|\,)', txt)
    r = [_ for _ in r if len(_)>1 and 'http' not in _]
    return r


In [192]:
corpus = []
tokens = Counter()
for q in tqdm(questions):
    corpus.append(process_textblob(q[0]))
    for a in q[2]:   
        r = process_textblob("".join(a[1]))
        corpus.append(r)
        #tokens |= Counter(r)
#len(tokens)

100%|██████████| 124256/124256 [00:34<00:00, 3654.42it/s]


In [193]:
corpus[-1]

['страдающее',
 'средневековье',
 'для',
 'comsouffrantmittelalterдилетант',
 'просто',
 'отличный',
 'исторический',
 'паблик',
 'неплохой',
 'comdiletantmedia']

In [194]:
from gensim import models

In [195]:
word2vec_size=512

In [196]:
w2v = models.Word2Vec(corpus, min_count=1, size=word2vec_size)

In [197]:
def fill_sentance_matrix(sent):
    matrix = []
    for i, w in enumerate(process_textblob(sent)):
        matrix.append(w2v[w])
    return np.matrix(matrix)

In [198]:
epochs = 0

In [370]:
topics = set()
topics_to_id = {}
for q in questions:
    topics|=set([_.lower() for _ in q[1]])
for i,t in enumerate(topics):
    topics_to_id[t] = i

In [373]:
question_postprocessed = []
answers_postprocessed = []
question_ids = []
answers_ids = []
for i, q in tqdm(enumerate(questions)):
    # print(q)
    #print((len(q['answers']), len(q['text']), word2vec_size))
    future_answers = []
    for j, a in enumerate(q[2]):
        r = fill_sentance_matrix("".join(a[1]))
        if r.shape[1]==word2vec_size:
            future_answers.append(len(answers_postprocessed))
            answers_postprocessed.append(r)
            answers_ids.append((i, j))
    if len(future_answers)>0:
        question_postprocessed.append((fill_sentance_matrix(q[0]), future_answers))
        question_ids.append(i)
    for t in q[1]:
        question_topics[i,topics_to_id[t.lower()]] = 1
    


124256it [01:44, 1189.69it/s]


In [388]:
questions[35][2][13]

['Мама на втором круге декрета',
 'Анна, а у вас прямо-таки оригинальный рецепт болоньезе? Без морковки и сельдерея, не говоря уже о других ингредиентах =)']

In [438]:
answer_topics = np.zeros((len(answers_ids), len(topics_to_id)))
for i,a in tqdm(enumerate(answers_ids)):
    for t in questions[a[0]][1]:
        answer_topics[i, topics_to_id[t.lower()]] = 1
question_topics = np.zeros((len(question_ids), len(topics_to_id)))
for i,a in tqdm(enumerate(question_ids)):
    for t in questions[a][1]:
        question_topics[i,topics_to_id[t.lower()]] = 1


231704it [00:03, 60174.79it/s] 
94144it [00:00, 120627.02it/s]


In [265]:
print(max((_ for _ in enumerate(question_postprocessed)), key=lambda _: len(_[1][1])))

(21136, (matrix([[ 1.49883032, -0.98222858,  2.50586653, ..., -0.4661651 ,
          0.39683902, -0.47477928],
        [-0.25660571, -0.27273744,  0.07351503, ...,  0.11165854,
          0.11696509, -0.00326384],
        [-0.10612646, -0.62579906,  0.21758351, ...,  1.73073304,
          1.12295938,  0.01356338],
        ..., 
        [-0.40986386,  0.83124447,  0.81553257, ..., -0.36933964,
         -0.22748461, -0.72318798],
        [ 0.80690885,  0.9055205 ,  0.89453435, ...,  0.53179806,
         -0.5272482 ,  1.16792071],
        [ 0.4561269 ,  1.12210906,  0.35627127, ...,  0.4817827 ,
          2.01626706, -0.25266549]], dtype=float32), [53085, 53086, 53087, 53088, 53089, 53090, 53091, 53092, 53093, 53094, 53095, 53096, 53097, 53098, 53099, 53100, 53101, 53102, 53103, 53104, 53105, 53106, 53107, 53108, 53109, 53110, 53111, 53112, 53113, 53114, 53115, 53116, 53117, 53118, 53119, 53120, 53121, 53122, 53123, 53124, 53125, 53126, 53127, 53128, 53129, 53130, 53131, 53132, 53133, 5313

In [269]:
question_ids[21136]
#questions[29252]

29252

In [202]:
input_questions = T.tensor3('input sequence','float32')
input_positive_answers = T.tensor3('input sequence',"float32")
input_negative_answers = T.tensor3('input sequence','float32')

In [416]:
output_topics = T.matrix('batch topics', 'float32')

In [203]:
target_space_n = 100

In [222]:
class question:
    l = lasagne.layers.InputLayer(shape=(None, None, word2vec_size),input_var=input_questions)
    l = lasagne.layers.LSTMLayer(l, 200,  only_return_final=True)
    out = lasagne.layers.DenseLayer(l, target_space_n, nonlinearity=None)

In [223]:
class answer:
    l  = lasagne.layers.InputLayer(shape=(None, None, word2vec_size))
    l = lasagne.layers.LSTMLayer(l, 200,  only_return_final=True)
    out = lasagne.layers.DenseLayer(l, target_space_n, nonlinearity=None)


In [419]:
 len(tokens)

0

In [539]:
class question_topics_model:
    weights = []
    l = lasagne.layers.DenseLayer(question.out, 300)
    weights += [l.W, l.b]
    out = lasagne.layers.DenseLayer(l, len(topics), nonlinearity=lasagne.nonlinearities.sigmoid)
    weights += [out.W, out.b]
    
    
questions_topic_out = lasagne.layers.get_output(question_topics_model.out, inputs=input_questions)

In [540]:
question_topics_model.out.W.get_value()

array([[ -1.03813279e-02,  -1.71564519e-02,  -2.45445017e-02, ...,
         -2.55452804e-02,  -2.94079585e-03,  -1.90537516e-02],
       [  1.85249075e-02,  -1.88941825e-02,   5.98762790e-03, ...,
          1.14345020e-02,  -5.84423309e-03,   7.48844072e-03],
       [ -1.24681145e-02,   2.01238040e-02,  -2.49961875e-02, ...,
         -1.45954015e-02,  -5.56790596e-03,  -5.83296921e-03],
       ..., 
       [ -1.37532176e-02,  -4.23294399e-03,   1.80464406e-02, ...,
         -1.31168384e-02,  -3.84850646e-05,  -1.94346886e-02],
       [  2.53016725e-02,  -1.43302567e-02,   2.00576521e-02, ...,
          2.10965574e-02,   3.71659524e-04,  -2.60295020e-03],
       [  1.53620560e-02,   1.62013732e-02,   1.73143204e-02, ...,
         -6.69651432e-03,  -1.81789864e-02,  -5.09035634e-03]], dtype=float32)

In [541]:
questions_vec = lasagne.layers.get_output(question.out, inputs=input_questions)
positive_answers_vec = lasagne.layers.get_output(answer.out, inputs=input_positive_answers)
negative_answers_vec = lasagne.layers.get_output(answer.out, inputs=input_negative_answers)

AttributeError: 'function' object has no attribute 'out'

In [542]:
loss_topics = lasagne.objectives.binary_crossentropy(questions_topic_out, output_topics).mean()

updates_topics = lasagne.updates.adam(loss_topics, question_topics_model.weights)

In [555]:
questions_topic_out.eval({input_questions:X[0:1]}).argmax()

6394

In [544]:
#обучение
train_topics = theano.function([input_questions, output_topics], loss_topics, updates=updates_topics)

#функция потерь без обучения

compute_cost_topics = theano.function([input_questions, output_topics], loss_topics)


INFO (theano.gof.compilelock): Refreshing lock /root/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-3.5.2-64/lock_dir/lock
INFO:theano.gof.compilelock:Refreshing lock /root/.theano/compiledir_Linux-4.4--generic-x86_64-with-debian-stretch-sid-x86_64-3.5.2-64/lock_dir/lock


In [556]:
predict_topic = theano.function([input_questions], questions_topic_out)

In [557]:
predict_topic(X)

array([[  4.29366220e-04,   4.06810759e-05,   1.21993298e-05, ...,
          1.75588320e-05,   1.22693658e-04,   1.58121242e-04],
       [  7.60033363e-05,   7.62892159e-05,   9.61311162e-06, ...,
          5.59203727e-05,   5.19639626e-03,   3.98258780e-05],
       [  5.15145133e-04,   4.09953645e-06,   6.53908455e-06, ...,
          6.14609453e-05,   5.54156548e-04,   6.24919252e-04],
       ..., 
       [  8.67512881e-06,   8.42127201e-10,   7.81661171e-08, ...,
          8.02863980e-08,   7.01328190e-06,   2.36374945e-05],
       [  3.04347304e-05,   2.07446146e-06,   4.59525518e-06, ...,
          3.15521407e-04,   1.68966446e-02,   7.93410800e-05],
       [  4.55919762e-05,   3.30826602e-08,   8.51024288e-06, ...,
          1.94750792e-05,   6.67592231e-03,   4.45627738e-05]], dtype=float32)

In [549]:
batch_size_topics = 100
def iterate_minibatches_topics(input_q, output_topics, test = False):
    if test:
        fr = int(len(input_q)*0.8)
        to = int(len(input_q))
    else:
        fr = int(0)
        to = int(len(input_q)*0.8)
        
    for i in range(fr,to,batch_size_topics):
        arr = []
        for a in input_q[i:i+batch_size_topics]:
            arr.append(a[0])
        yield prepare_batch(arr), output_topics[i:i+batch_size_topics]
for i in range(50):       
    for X, Y in iterate_minibatches_topics(question_postprocessed, question_topics):
        #print(X.shape, Y.shape)
        (train_topics(X, Y.astype('float32')))
    print("Train score at %s" % i, compute_cost_topics(X, Y.astype('float32')))
    for X, Y in iterate_minibatches_topics(question_postprocessed, question_topics, test=True):
        print("Test score at %s" % i, compute_cost_topics(X, Y.astype('float32')))
        break


Train score at 0 0.002233854727819562
Test score at 0 0.0022714207880198956
Train score at 1 0.0021669859997928143
Test score at 1 0.002215204993262887
Train score at 2 0.0021167038939893246
Test score at 2 0.002172712469473481
Train score at 3 0.0020732900593429804
Test score at 3 0.0021353322081267834
Train score at 4 0.0020334827713668346
Test score at 4 0.002101243007928133


KeyboardInterrupt: 

In [524]:
X.shape, Y.shape

((100, 24, 512), (100, 8866))

In [226]:
alpha = 3
print(questions_vec)
loss = T.nnet.relu(((questions_vec - positive_answers_vec)**2).sum(axis=1) - 
              ((questions_vec - negative_answers_vec)**2).sum(axis=1) + alpha).mean()
#loss = (questions_vec + negative_answers_vec + positive_answers_vec).mean()

Elemwise{add,no_inplace}.0


In [227]:
updates = lasagne.updates.adadelta(loss, weights,)

In [228]:

#обучение
train = theano.function([input_questions, input_positive_answers, input_negative_answers], loss, updates=updates)

#функция потерь без обучения
compute_cost = theano.function([input_questions, input_positive_answers, input_negative_answers], loss)


In [229]:
transform_answer = theano.function([input_positive_answers], positive_answers_vec, allow_input_downcast=True)
transform_question = theano.function([input_questions], questions_vec, allow_input_downcast=True)

In [230]:
import random
random.shuffle(question_postprocessed)

In [235]:
batch_size = 100
num_epochs = 500
def iterate_minibatches(inputs, test = False):
    if not test:
        for i in range(0, int(len(inputs)*0.8), batch_size):
            yield inputs[i:i+batch_size]
    else:
        for i in range(int(len(inputs)*0.8), len(inputs), batch_size):
            yield inputs[i:i+batch_size]

In [452]:
def prepare_batch(arr, size=None):
    
    if size==None:
        size = 0
        for i in arr:
            size = max(size, i.shape[0])
    ret = np.zeros((len(arr), size, word2vec_size), dtype='float32')
    for i,e in enumerate(arr):
        if e.shape[1]!=word2vec_size:
             ret[i] = np.zeros((size,word2vec_size))
        else:
            ret[i] = np.concatenate((np.zeros((size-e.shape[0],word2vec_size)), e), axis=0)
    return ret

In [233]:
def get_batch_inputs(batch):
    max_len_pos = 0
    current_questions = []
    positive = []
    negative = []
    questions_size = 0
    positive_size = 0
    negative_size = 0
    for question in batch:
        e = answers_postprocessed[random.choice(question[1])]
        current_questions.append(question[0])
        questions_size = max(questions_size, question[0].shape[0])
        positive.append(e)
        positive_size = max(positive_size, e.shape[0])
        e = random.randint(0, len(answers_postprocessed)-1)
        while e in question[1]:
            e = random.randint(0, len(answers_postprocessed))
        e  = answers_postprocessed[e]
        negative.append(e)
        negative_size = max(negative_size, e.shape[0])


    current_questions = a(current_questions, questions_size)
    positive = prepare_batch(positive, positive_size)
    negative = prepare_batch(negative, negative_size)
    return current_questions, positive, negative

In [ ]:
for i in range(num_epochs):
    for batch in iterate_minibatches(question_postprocessed):
        current_questions, positive, negative = get_batch_inputs(batch)
        
        if random.random()<0.01:
            print(train(current_questions, positive, negative))
        
    print("Epoch %s. Train:" % i, compute_cost(current_questions, positive, negative))
    for batch in iterate_minibatches(question_postprocessed, True):
        current_questions, positive, negative = get_batch_inputs(batch)
        break
        
    print("Epoch %s. Test" % i, compute_cost(current_questions, positive, negative))


2.880577802658081
0.7332930564880371


KeyboardInterrupt: 

In [486]:
for batch in iterate_minibatches(question_postprocessed, True):
    current_questions, positive, negative = get_batch_inputs(batch)
    break

print("Epoch %s. Test" % i, compute_cost(current_questions, positive, negative))


Epoch 6. Test 0.8835563659667969


In [255]:
weight = lasagne.layers.get_all_param_values(question.out)
np.save('weights/question.npy', weight)
weight = lasagne.layers.get_all_param_values(answer.out)
np.save('weights/answer.npy', weight)

In [254]:
!mkdir weights

In [256]:
w2v.save('weights/w2v')

In [318]:
len(answers_vectors)

1

In [319]:
answers_vectors = []
for i, value in tqdm(zip(answers_ids, answers_postprocessed)):
    a = value.copy()
    a.resize((1,) + value.shape)
    answers_vectors.append(transform_answer(a)[0])

231704it [16:41, 231.40it/s]


In [571]:
def answer(st, rand=False):
    if rand:
        rand_id = random.randint(0, len(question_ids))
        st = questions[question_ids[rand_id]][0] 
    
    a= fill_sentance_matrix(st).copy()
    a.resize((1,) + value.shape)
    vector = transform_question(a)[0]
    min_id = 0
    min_dist = 10**9
    min_ans = 0
    t = predict_topic(a)[0]
    for num in tqdm(range(len(answers_ids))):
        i, ans = answers_ids[num], answers_vectors[num] 
        d = ((ans-vector)**2).sum()
        if rand:
            d-=answer_topics[num].dot(question_topics[rand_id])
        else:
            d-=answer_topics[num].dot(t)
        if d<min_dist:
            min_dist = d
            min_ans = ans
            min_id = i
            #print(d)
    #print(vector)
    #print(min_ans)
    print(min_dist)
    #print(min_id)
    if rand:
        print(st)
    
    #print(questions[min_id[0]])
    print(questions[min_id[0]][2][min_id[1]-1])
#answer('', rand=True)

In [584]:
answer('', rand=True)

100%|██████████| 231704/231704 [00:02<00:00, 79166.33it/s]

4.58096885681
Считаете ли вы нормальным, что в большинстве общеобразовательных школ пропал дресс-код?
['копирайтер, фотограф', 'А что тут ненормального? Заставлять 11 лет носить то, что тебе не нравится, не идет и всячески тебя расстраивает - это нисколько не уравнение богатых и бедных. Если тебе удобно и ты хорошо себя чувствуешь в своем "свитшоте" и "джеггинсах", простигосподи, то твоя голова не будет весь день занята тем, что с тобой что-то не так, юбка велика, кофта дурацкая и пиджак отстойного цвета. И вполне спокойно можно думать про ЕГЭ, мальчиков и сочинение по русскому.\xa0И никаких конфликтов из-за одежды ни у кого не возникает. Уже не в совке живем, где кофта, привезенная папой-военным из другой страны, вызывает кучу ахов и сплетен.\xa0']


In [588]:
answer('Куда съездить в путешествие?')

100%|██████████| 231704/231704 [00:04<00:00, 49643.63it/s]

6.29681209542
['студентка социологии', 'Мой любимый маршрут.От метро Курская, через Воронцово Поле до Покровки. Дальше вверх по Покровскому бульвару и по Чистопрудному. Выходите на Мясницкую Улицу (если всать спиной к м. Чистые пруды так, чтоб перед тобой была Мясницкая улица, то надо идти налево по ней) - здесь знаменитый очень красивый Чайный Дом и Главпочтамт. Пройдя рпактически всю улицу до Домика Алисы (голубенький домик по левой стороне). в Братьях Караваевых можно прикупить вкусненьких пирожных. Мясницкая улица после реконструкции прошлым летом просто чудесная сейчас. Повернуть на Армянский переулок, дальше по нему пересечь Маросейку и продолжать идти вниз - по Старосадскому. Там можно долго гулять по переулкам), зайти в Морозовский садик (я зову его "секретный", он малоизвестный и неприметный). Дойти до Высотки на Котельнической набережной, обойти её не со стороны набережной, а внутри - там красота. Перейти по большому Устьинскому мосту через Москва-реку на Болотный остров, не 

In [333]:
random.choice(questions)

['Почему у электричек есть нелюбимые станции? Везде останавливается, а на 43 километре, блин, нет!',
 ['Как это работает?', 'Зачем я это узнал?'],
 []]

In [311]:
np.concatenate((a, a), axis = 0).shape

ValueError: shape too large to be a matrix.

matrix([[ 0.06272896,  0.24435805, -0.04783468, ..., -0.05889111,
          0.16657655, -0.24520613],
        [ 0.08878329,  0.79924482, -0.33646864, ...,  0.16268285,
          0.30213052, -0.79116642]], dtype=float32)

In [279]:
tmp = value.copy()

In [288]:
positive.shape

(100, 743, 512)

In [307]:
value.resize(( 111, 512))